In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import numpy as np
import keras
from keras.models import *
from keras.layers import *
from tensorflow.python.keras import losses
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint
import cv2
import glob
import matplotlib.pyplot as plt
import itertools
from util import losses, custom_data_generator, metrics 
from keras.utils import multi_gpu_model
from models import model_loader
import datetime
from processing import abs_sobel_thresh
from processing import mag_threshold
from processing import *
from util import custom_data_generator as data_util
from models.common import lanenet_wavelet

# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
# set_session(tf.Session(config=config))


batch_size =30
is_train =  False
# model_name = 'lanenet'
image_width,image_height = 512,512
channeles = 3
# checkpoint_name = "checkpoint_lanenet"
one_hot_label= False
data_aug = False

 





In [ ]:
# checkpoint_dir = "./checkpoints/%s/"%(checkpoint_name)

# path = '../../../Lane_Data/GAN_merged_v4/train_h'
# imgPath = './full_img.png'/
path = '/home/beemap/Documents/cesar-workspace/Lane_Data/Mix_dataset_full_v2/val'
val_inputs_path = path +"/"
val_masks_path = path +"_labels/"


val_samples = glob.glob(val_inputs_path + "*")

print("Validation samples = %s\n\n"%(len(val_samples)))


In [ ]:
print(len(val_samples))

In [ ]:
img = cv2.imread(val_samples[-1], -1)
img = cv2.imread(imgPa)
#img = cv2.resize(img, (256, 256))
img.shape
#plt.imshow(img)

In [ ]:
imgPath = './full_img.png'
BigImg = cv2.imread(imgPath, -1)
BigImg.shape

In [ ]:
maskPath = './mask.png'
BigMask = cv2.imread(maskPath, 0)
index = BigMask == 0
BigImg[index] = (0, 0, 0)

In [ ]:
new_dir = './predictions_lanes/'
if not os.path.isdir(new_dir):
    os.makedirs(new_dir)
 


In [ ]:
# checkpoint_name = "checkpoint_lanenet"
checkpoint_name = "Mix_dataset_full_v2_lanenet"

model_name = 'lanenet'
checkpoint_dir = "./checkpoints/%s/"%(checkpoint_name)

if is_train:   
    
     print("Training .....")

else:
    print("Loading Model .... ",checkpoint_dir)
    json_file = open(checkpoint_dir+model_name+".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights("./checkpoints/%s/%s_weights_90.h5"%(checkpoint_name,model_name))
    #asd
#     print(checkpoint_path)
#     files = glob.glob('/home/beemap/Documents/noor-workspace/Semantic-Segmentation-Suite/airbus/train/*.png')
#     accu = []
#     BG_IU, BD_IU, BG_P, BD_P = metrics.calculate_IoU_Per_Epoch(model,val_inputs_path,val_masks_path,checkpoint_dir,1,False)
#     for i in range(50):
#         gen = next(train_generator)
#     gen = next(train_generator)

In [ ]:
def binary_pipeline(img, mask):
    
    ind = (mask == 0)
    img[ind] = 0
    img_copy = cv.GaussianBlur(img, (3, 3), 0)
    #img_copy = np.copy(img)
    
    # color channels
    s_binary = hls_select(img_copy, sthresh=(140, 255), lthresh=(120, 255))
    #red_binary = red_select(img_copy, thresh=(200,255))
    
    # Sobel x
    x_binary = abs_sobel_thresh(img_copy,thresh=(25, 200))
    y_binary = abs_sobel_thresh(img_copy,thresh=(25, 200), orient='y')
    xy = cv.bitwise_and(x_binary, y_binary)
    
    #magnitude & direction
    mag_binary = mag_threshold(img_copy, sobel_kernel=3, thresh=(30,100))
    dir_binary = dir_threshold(img_copy, sobel_kernel=3, thresh=(0.8, 1.2))
    
    # Stack each channel
    gradient = np.zeros_like(s_binary)
    gradient[((x_binary == 1) & (y_binary == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    final_binary = cv.bitwise_or(s_binary, gradient)
    
    return final_binary

In [ ]:
from keras.callbacks import TensorBoard,Callback
from PIL import Image
import shutil
import matplotlib.pyplot as plt

n = 29
m = 20

i = 0
for image in val_samples[:]:
    full_img = cv2.imread(image, 1)#[:, 256:, :3]
    img_ = full_img[:, :256, :3]
#     segmented = full_img[:, 512:, :]
    h, w,_ = img_.shape
#     img_ = cv2.resize(img_, (512, 512))

    tmp = img_[:]
#     img_ = cv2.resize(img_, (256, 256))
    r_img = img_
    img_ = np.float32(img_)/255.0
    name = os.path.basename(image)
    maskPath = val_masks_path + name#[3:]
    gt = cv2.imread(maskPath, -1)
    if img_.shape[0] ==(512):
        image_list = []
        for n in range(0, 512, 256):
            for m in range(0, 512, 256):
                image_list.append(img_[n:n+256, m:m+256, :])
    else:
        image_list = [img_]
    seg_list = []
    name = os.path.basename(image)      
    for i, img in enumerate(image_list):
#         input_image_gray = data_util.get_image(image,do_aug=[],gray=True, change=False)
        input_image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        w1, w2, w3, w4 = lanenet_wavelet(input_image_gray)
        w1 = np.expand_dims(w1, axis=0)
        w2 = np.expand_dims(w2, axis=0)
        w3 = np.expand_dims(w3, axis=0)
        w4 = np.expand_dims(w4, axis=0)

        mask = model.predict([np.expand_dims(img, axis=0), w1, w2, w3, w4], batch_size=None, verbose=0, steps=None)
        mask = np.round(mask[0, :, :, 0]).astype(int)

        seg = np.zeros((256, 256, 3))

        seg[:, :, 0] += ((mask[:, :] == 1) * (255)).astype('uint8')
        seg[:, :, 1] += ((mask[:, :] == 1) * ( 255)).astype('uint8')
        seg[:, :, 2] += ((mask[:, :] == 1) * ( 255)).astype('uint8')
        seg_list.append(seg)
    final_seg = np.zeros((h, w, 3))
    for i,seg in enumerate(seg_list):
        if i == 0:
            final_seg[:256, :256, :] = seg
        elif i == 1:
            final_seg[:256, 256:, :] = seg
        elif i == 3:
            final_seg[256:, 256:, :] = seg
        else:
            final_seg[256:, :256, :] = seg
            
    dest = new_dir + name
#     index = np.all((segmented == np.array((0, 0, 0)).reshape(1, 1, 3)), axis=2)
#     final_seg[index] = np.array((0, 0, 0))
#         r = 0 if i < 2 else 1
#         c = 0 if i % 2 == 0 else 1
        
#         bigImg[r*256 : (r+1)*256, c*256 : (c+1)*256, :] = img*255
#         bigMask[r*256 : (r+1)*256, c*256 : (c+1)*256, :] = seg
#         filename = new_dir + name
#     seg = cv2.resize(seg, (512, 512))
#         cv2.imwrite(filename, np.concatenate((bigImg, bigMask), axis=1))
    merge = np.concatenate((full_img, gt, final_seg), axis=1)
#         pred_list.append(merge)
    cv2.imwrite(dest, merge)
    



In [ ]:
from keras.callbacks import TensorBoard,Callback
from PIL import Image
import shutil
import matplotlib.pyplot as plt

BigLane = np.zeros_like((BigImg))
for r in range(0, BigImg.shape[0], 256):
    for c in range(0, BigImg.shape[1], 256):
        
        img = BigImg[r:r+256, c:c+256, :3]
#         h, w, _ = img.shape
        tmp = img[:]
        img = np.float32(img)/255.0
        
#         input_image_gray = data_util.get_image(image,do_aug=[],gray=True, change=False)
        input_image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        w1, w2, w3, w4 = lanenet_wavelet(input_image_gray)
        w1 = np.expand_dims(w1, axis=0)
        w2 = np.expand_dims(w2, axis=0)
        w3 = np.expand_dims(w3, axis=0)
        w4 = np.expand_dims(w4, axis=0)

        mask = model.predict([np.expand_dims(img, axis=0), w1, w2, w3, w4], batch_size=None, verbose=0, steps=None)

        mask = np.round(mask[0, :, :, 0]).astype(int)
        seg = np.zeros((256, 256, 3))

        seg[:, :, 0] += ((mask[:, :] == 1) * (255)).astype('uint8')
        seg[:, :, 1] += ((mask[:, :] == 1) * ( 255)).astype('uint8')
        seg[:, :, 2] += ((mask[:, :] == 1) * ( 255)).astype('uint8')
        
        BigLane[r:r+256, c:c+256, :] = seg
name = 'BigLane2.png' 
new_dir = './'
dest = new_dir + name
cv2.imwrite(dest, BigLane)




In [ ]:
Image.fromarray(BigLane)

In [ ]:
print("\nBackground IOU = %02f"%BG_IU)
print("Main-Class IOU = %02f"%BD_IU)
print("Mean IOU = %02f"%((BG_IU + BD_IU)/2))
print("Background P-Accuracy = %02f"%BG_P)
print("Main-Class P-Accuracy = %02f\n"%BD_P)